In [1]:
import requests
import time
import json
import pymysql

In [2]:
host = "localhost"
user = "root"
db_name = 'dapp_analysis_rearrange'

db = pymysql.connect(host = host, user = user, db = db_name)
cursor = db.cursor()

In [3]:
url = "https://www.4byte.directory/api/v1/signatures/?page="

In [4]:
def get_data(page):
    while True:
        req = requests.get(url + str(page))
        if req.status_code == 404:
            return None
        
        if req.status_code == 200:
            data = req.json()
            return data["results"]
        
        time.sleep(180) # meet rate limit

In [6]:
query = "INSERT INTO methodabi(id, text_signature, hex_signature, bytes_signature, created_at) VALUES (%s,%s,%s,%s,%s)"

for page in range(1, 10514):
    data = get_data(page)
    if data is None:
        print(f"Done, collected data from {page} pages")
        break
    
    args = (data["id"], data["text_signature"], data["hex_signature"], data["bytes_signature"], data["created_at"],)
    cursor.execute(query, args)
    db.commit()

Done, collected data from 10513 pages
